In [ ]:
from dash import Dash, html, dcc, dash_table
import pandas as pd
from dash.dependencies import Input, Output, State, ALL
import dash_bootstrap_components as dbc
import umap.umap_ as umap
from sklearn.cluster import KMeans
import plotly.graph_objects as go

app = Dash(__name__)
#---------------------------------------------------------------------------------------------------
path = 'path_to_input_file.csv'
df = pd.read_csv(path, na_values='NULL')
df.set_index('your_index', inplace=True)
df['your_index'] = range(1, len(df) + 1)
PAGE_SIZE = 5
total_pages = len(df) // PAGE_SIZE
if len(df) % PAGE_SIZE != 0:
    total_pages += 1

# Set the page_current to the last page if there is data
page_current = max(total_pages - 1, 0)
# Create input fields for adding new rows
new_row_inputs = []
#---------------------------------------------------------------------------------------------------
table_mutations = dash_table.DataTable(
    id='datatable-paging',
    columns=[],  # Empty columns initially
    page_current=page_current,
    page_size=PAGE_SIZE,
    page_action='custom',
    editable=True,  # Allow editing of the table
    row_deletable=True,  # Allow deletion of rows
)

checkbox_categories = {
    "Clinical and laboratory values": ['AGE', 'SEX', 'FEV', 'EXAML', 'WBC', 'HB', 'PLT', 'LDH', 'BMB', 'PBB'],
    "Cytogenetics": ['CGNK', 'CGCX', 'inv16t16.16', 't8.21', 't.6.9..p23.q34.', 'inv.3..q21.q26.2.',
                  'minus.5', 'del.5q.', 't.9.22..q34.q11.', 'minus.7', 'minus.17', 't.v.11..v.q23.',
                  'abn.17p.', 'inv.16.', 't.9.11..p21.23.q23.', 't.8.21.'],
    "Molecular genetics": ['ASXL1', 'ATRX', 'BCOR', 'BCORL1', 'BRAF', 'CALR', 'CBL', 'CBLB', 'CDKN2A', 'CEBPA',
                  'CEBPADM', 'CEBPA.bZIP', 'CEBPA.bZIP.inframe', 'CEBPA.TAD', 'CEBPASTAT', 'CSF3R', 'CUX1',
                  'DNMT3A', 'ETV6', 'EZH2', 'FBXW7', 'FLT3I', 'FLT3T', 'FLT3R', 'GATA1', 'GATA2', 'GNAS',
                  'HRAS', 'IDH1', 'IDH2', 'IKZF1', 'JAK2', 'KDM6A', 'KIT', 'KRAS', 'MPL', 'MYD88',
                  'NOTCH1', 'NPM1', 'NRAS', 'PDGFRA', 'PTEN', 'PHF6', 'PTPN11', 'RAD21', 'RUNX1', 'SETBP1',
                  'SF3B1', 'SMC1A', 'SMC3', 'SRSF2', 'STAG2', 'TET2', 'TP53', 'U2AF1', 'WT1', 'ZRSR2']
}

# Create a list of all features for the header ordering
all_features = [feature for sublist in checkbox_categories.values() for feature in sublist]
# Create input fields for adding new rows
new_row_inputs = []

checklist_mutations = html.Div([
    html.H5("Clinical and laboratory values"),
    dcc.Checklist(
        id="select_feature_group_1",
        options=[{'label': html.Label('AGE', title='Age of the patients in years'), 'value': 'AGE', 'disabled': False},
                 {'label': html.Label('SEX', title='Sex of the patient: 0 for male, 1 for female'), 'value': 'SEX', 'disabled': False},
                 {'label': html.Label('FEV', title='FEV transcription factor, ETS family member'), 'value': 'FEV', 'disabled': False},
                 {'label': html.Label('EXAML', title=''), 'value': 'EXAML', 'disabled': False},
                 {'label': html.Label('WBC', title='White blood cells'), 'value': 'WBC', 'disabled': False},
                 {'label': html.Label('HB', title='Hemoglobin'), 'value': 'HB', 'disabled': False},
                 {'label': html.Label('PLT', title='Platelet count'), 'value': 'PLT', 'disabled': False},
                 {'label': html.Label('LDH', title='Lactate dehydrogenase'), 'value': 'LDH', 'disabled': False},
                 {'label': html.Label('BMB', title='Bone marrow biopsies'), 'value': 'BMB', 'disabled': False},
                 {'label': html.Label('PBB', title='Polybrominated Biphenyls'), 'value': 'PBB', 'disabled': False}
                 ],
        value=['AGE', 'PBB', 'BMB', 'PLT'],
        labelStyle=dict(display='inline')
    ),
    html.H5("Cytogenetics"),
    dcc.Checklist(
        id="select_feature_group_2",
        options=[{'label': x, 'value': x, 'disabled': False} for x in checkbox_categories["Cytogenetics"]],
        value=[],
        labelStyle=dict(display='inline')
    ),
    html.H5("Molecular genetics"),
    dcc.Checklist(
        id="select_feature_group_3",
        options=[{'label': html.Label('ASXL1', title='ASXL transcriptional regulator 1'), 'value': 'ASXL1', 'disabled': False},
                 {'label': html.Label('ATRX', title='ATRX chromatin remodeler'), 'value': 'ATRX', 'disabled': False},
                 {'label': html.Label('BCOR', title='BCL6 corepressor'), 'value': 'BCOR', 'disabled': False},
                 {'label': html.Label('BCORL1', title='BCL6 corepressor like 1'), 'value': 'BCORL1', 'disabled': False},
                 {'label': html.Label('BRAF', title='B-Raf proto-oncogene, serine/threonine kinase'), 'value': 'BRAF', 'disabled': False},
                 {'label': html.Label('CALR', title='calreticulin'), 'value': 'CALR', 'disabled': False},
                 {'label': html.Label('CBL', title='Cbl proto-oncogene'), 'value': 'CBL', 'disabled': False},
                 {'label': html.Label('CBLB', title='Cbl proto-oncogene B'), 'value': 'CBLB', 'disabled': False},
                 {'label': html.Label('CDKN2A', title='cyclin dependent kinase inhibitor 2A'), 'value': 'CDKN2A', 'disabled': False},
                 {'label': html.Label('CEBPA', title='CCAAT enhancer binding protein alpha'), 'value': 'CEBPA', 'disabled': False},
                 {'label': html.Label('CEBPADM', title=''), 'value': 'CEBPADM', 'disabled': False},
                 {'label': html.Label('CEBPA.bZIP', title=''), 'value': 'CEBPA.bZIP', 'disabled': False},
                 {'label': html.Label('CEBPA.bZIP.inframe', title=''), 'value': 'CEBPA.bZIP.inframe', 'disabled': False},
                 {'label': html.Label('CEBPA.TAD', title=''), 'value': 'CEBPA.TAD', 'disabled': False},
                 {'label': html.Label('CEBPASTAT', title=''), 'value': 'CEBPASTAT', 'disabled': False},
                 {'label': html.Label('CSF3R', title='colony stimulating factor 3 receptor'), 'value': 'CSF3R', 'disabled': False},
                 {'label': html.Label('CUX1', title='cut like homeobox 1'), 'value': 'CUX1', 'disabled': False},
                 {'label': html.Label('DNMT3A', title='DNA methyltransferase 3 alpha'), 'value': 'DNMT3A', 'disabled': False},
                 {'label': html.Label('ETV6', title='ETS variant transcription factor 6'), 'value': 'ETV6', 'disabled': False},
                 {'label': html.Label('EZH2', title='enhancer of zeste 2 polycomb repressive complex 2 subunit'), 'value': 'EZH2', 'disabled': False},
                 {'label': html.Label('FBXW7', title='F-box and WD repeat domain containing 7'), 'value': 'FBXW7', 'disabled': False},
                 {'label': html.Label('FLT3I', title=''), 'value': 'FLT3I', 'disabled': False},
                 {'label': html.Label('FLT3T', title=''), 'value': 'FLT3T', 'disabled': False},
                 {'label': html.Label('FLT3R', title=''), 'value': 'FLT3R', 'disabled': False},
                 {'label': html.Label('GATA1', title='GATA binding protein 1'), 'value': 'GATA1', 'disabled': False},
                 {'label': html.Label('GATA2', title='GATA binding protein 2'), 'value': 'GATA2', 'disabled': False},
                 {'label': html.Label('GNAS', title='GNAS complex locus'), 'value': 'GNAS', 'disabled': False},
                 {'label': html.Label('HRAS', title='HRas proto-oncogene, GTPase'), 'value': 'HRAS', 'disabled': False},
                 {'label': html.Label('IDH1', title='isocitrate dehydrogenase (NADP(+)) 1'), 'value': 'IDH1', 'disabled': False},
                 {'label': html.Label('IDH2', title='isocitrate dehydrogenase (NADP(+)) 2'), 'value': 'IDH2', 'disabled': False},
                 {'label': html.Label('IKZF1', title='IKAROS family zinc finger 1'), 'value': 'IKZF1', 'disabled': False},
                 {'label': html.Label('JAK2', title='Janus kinase 2'), 'value': 'JAK2', 'disabled': False},
                 {'label': html.Label('KDM6A', title='lysine demethylase 6A'), 'value': 'KDM6A', 'disabled': False},
                 {'label': html.Label('KIT', title='KIT proto-oncogene, receptor tyrosine kinase'), 'value': 'KIT', 'disabled': False},
                 {'label': html.Label('KRAS', title='KRAS proto-oncogene, GTPase'), 'value': 'KRAS', 'disabled': False},
                 {'label': html.Label('MPL', title='MPL proto-oncogene, thrombopoietin receptor'), 'value': 'MPL', 'disabled': False},
                 {'label': html.Label('MYD88', title='MYD88 innate immune signal transduction adaptor'), 'value': 'MYD88', 'disabled': False},
                 {'label': html.Label('NOTCH1', title='notch receptor 1'), 'value': 'NOTCH1', 'disabled': False},
                 {'label': html.Label('NPM1', title='nucleophosmin 1'), 'value': 'NPM1', 'disabled': False},
                 {'label': html.Label('NRAS', title='NRAS proto-oncogene, GTPase'), 'value': 'NRAS', 'disabled': False},
                 {'label': html.Label('PDGFRA', title='platelet derived growth factor receptor alpha'), 'value': 'PDGFRA', 'disabled': False},
                 {'label': html.Label('PTEN', title='phosphatase and tensin homolog'), 'value': 'PTEN', 'disabled': False},
                 {'label': html.Label('PHF6', title='PHD finger protein 6'), 'value': 'PHF6','disabled': False},
                 {'label': html.Label('PTPN11', title='protein tyrosine phosphatase non-receptor type 11'), 'value': 'PTPN11', 'disabled': False},
                 {'label': html.Label('RAD21', title='RAD21 cohesin complex component'), 'value': 'RAD21', 'disabled': False},
                 {'label': html.Label('RUNX1', title='RUNX family transcription factor 1'), 'value': 'RUNX1', 'disabled': False},
                 {'label': html.Label('SETBP1', title='SET binding protein 1'), 'value': 'SETBP1', 'disabled': False},
                 {'label': html.Label('SF3B1', title='splicing factor 3b subunit 1'), 'value': 'SF3B1', 'disabled': False},
                 {'label': html.Label('SMC1A', title='structural maintenance of chromosomes 1A'), 'value': 'SMC1A', 'disabled': False},
                 {'label': html.Label('SMC3', title='structural maintenance of chromosomes 3'), 'value': 'SMC3', 'disabled': False},
                 {'label': html.Label('SRSF2', title='serine and arginine rich splicing factor 2'), 'value': 'SRSF2', 'disabled': False},
                 {'label': html.Label('STAG2', title='stromal antigen 2'), 'value': 'STAG2', 'disabled': False},
                 {'label': html.Label('TET2', title='tet methylcytosine dioxygenase 2'), 'value': 'TET2', 'disabled': False},
                 {'label': html.Label('TP53', title='tumor protein p53'), 'value': 'TP53', 'disabled': False},
                 {'label': html.Label('U2AF1', title='U2 small nuclear RNA auxiliary factor 1'), 'value': 'U2AF1', 'disabled': False},
                 {'label': html.Label('WT1', title='WT1 transcription factor'), 'value': 'WT1', 'disabled': False},
                 {'label': html.Label('ZRSR2', title='zinc finger CCCH-type, RNA binding motif and serine/arginine rich 2'), 'value': 'ZRSR2', 'disabled': False}
                 ],
        value=['DNMT3A', 'KIT', 'CGCX', 'NPM1'],
        labelStyle=dict(display='inline')
    )
])

UMAP = dcc.Graph(id='Umap_projection2d_live_adding', figure={})
dummy_div = html.Div(id='dummy-div', style={'display': 'none'})

app.layout = html.Div([
    html.H1("UMAP Projection for original data", title="With this application you can check to which risk group belong the specific patient.\n"
                                                       "You can mark any number of the variables and then write down your results.\n"
                                                       "Once you filled the cells with the information just press the button and wait for the results to appear.\n"
                                                       "If you want to update the results in the table just go to other page and come back.\n"
                                                       "If you do not know what the specific abbrevation mean, just hover on it",
            style={'text-align': 'left'}),
    checklist_mutations,
    table_mutations,
    html.H3("Enter new row data:"),
    html.Div(id='new-row-inputs-container'),  # Container for new row input fields
    dbc.Button('Add New Row', id='add-new-row-button', n_clicks=0, color='primary'),
    dcc.Graph(id='Umap_projection2d_live_adding', figure={}),  # Add the UMAP graph here
    dummy_div  # Include the dummy div in the layout
])

# Callback to dynamically update the new row input fields based on checklist selection
@app.callback(
    Output('new-row-inputs-container', 'children'),
    Input('select_feature_group_1', 'value'),
    Input('select_feature_group_2', 'value'),
    Input('select_feature_group_3', 'value'))

def update_new_row_inputs(group_1_selected, group_2_selected, group_3_selected):
    # Concatenate the selected features from all groups
    selected_features = group_1_selected + group_2_selected + group_3_selected
    # Create input fields for each selected feature with smaller cells
    new_row_inputs = [
        dbc.Input(
            id={'type': 'new-row-input', 'index': feature},
            type='text',
            placeholder=f'Enter {feature}',
            style={'width': '80px'}  # Adjust the width as desired
        )
        for feature in selected_features
    ]
    return new_row_inputs

# Callback to add a new row to the DataFrame when the "Add New Row" button is clicked
@app.callback(
    Output('dummy-div', 'children'),
    Input('add-new-row-button', 'n_clicks'),
    State({'type': 'new-row-input', 'index': ALL}, 'value'),
    State('select_feature_group_1', 'value'),
    State('select_feature_group_2', 'value'),
    State('select_feature_group_3', 'value'),
)
def add_new_row(n_clicks, new_row_values, group_1_selected, group_2_selected, group_3_selected):
    if n_clicks > 0:
        selected_features = group_1_selected + group_2_selected + group_3_selected
        num_features_selected = len(selected_features)
        num_input_values = len(new_row_values)
        if num_input_values == num_features_selected:
            # Create a dictionary for the new row data
            new_row = {col: val for col, val in zip(selected_features, new_row_values)}
            # Convert the new row to a DataFrame
            new_row_df = pd.DataFrame([new_row])
            # Concatenate the new row DataFrame to the existing DataFrame
            global df
            df = pd.concat([df, new_row_df], ignore_index=True)

            # Sort DataFrame in reverse order based on the index column (Pat) and reset the index
            df = df.sort_values(by='Pat', ascending=False).reset_index(drop=True)

    return None

@app.callback(
    Output('datatable-paging', 'data'),
    Output('datatable-paging', 'columns'),
    Input('datatable-paging', "page_current"),
    Input('datatable-paging', "page_size"),
    Input('select_feature_group_1', 'value'),
    Input('select_feature_group_2', 'value'),
    Input('select_feature_group_3', 'value'),
)
def update_table(page_current, page_size, group_1_selected, group_2_selected, group_3_selected):
    # Concatenate the selected features from all groups
    selected_features = group_1_selected + group_2_selected + group_3_selected
    # Filter the DataFrame based on selected features
    filtered_df = df[selected_features]
    # Generate columns dynamically based on selected features
    columns = [{"name": col, "id": col} for col in filtered_df.columns]

    start_index = page_current * page_size
    end_index = (page_current + 1) * page_size

    # Get the sliced data for the current page
    sliced_data = filtered_df.iloc[start_index:end_index].to_dict('records')

    return sliced_data, columns

@app.callback(
    Output('Umap_projection2d_live_adding', 'figure'),
    Input('select_feature_group_1', 'value'),
    Input('select_feature_group_2', 'value'),
    Input('select_feature_group_3', 'value'),
    Input('add-new-row-button', 'n_clicks'),  # Add the button as an input
)
def update_graph(group_1_selected, group_2_selected, group_3_selected, n_clicks):
    # Concatenate the selected features from all groups
    selected_features = group_1_selected + group_2_selected + group_3_selected
    # Filter the DataFrame based on selected features
    dff = df[selected_features]
    kmeans = KMeans(4)
    kmeans.fit(dff)
    clusters = dff.copy()
    clusters['clusters_pred'] = kmeans.fit_predict(dff)
    proj_2d = umap.UMAP(random_state=42).fit_transform(dff)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=proj_2d[:, 0],
        y=proj_2d[:, 1],
        mode='markers',
        marker=dict(
            size=3,
            color=clusters['clusters_pred'],
            colorscale='Viridis',
            showscale=True
        ),
        showlegend=False
    ))

    # Adding a red marker at the last point
    fig.add_trace(go.Scatter(
        x=[proj_2d[-1, 0]],
        y=[proj_2d[-1, 1]],
        mode='markers',
        marker=dict(size=10, color='red'),
        showlegend=False
    ))
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)